## 🛠️ **Primer prototipo o pruebas con datos**

En esta etapa necesitamos:

1. **Cargar** una oración de prueba.
2. **Definir** manualmente los parámetros del HMM (π, A, B).
3. **Preparar estructuras** para procesarlas en código.

### 📘 TEORÍA DE HMM (Modelo Oculto de Markov)

Según Rabiner:

- Un HMM es un modelo estadístico probabilístico en el que se asume que el sistema que se modela es un proceso de Markov con estados ocultos. donde:
    - El estado actual influye en los estados futuros, pero los estados en sí no son directamente observables.
    - Hay una secuencia de observaciones conocidas, lo que observamos son **emisiones** o **observaciones** que dependen del estado oculto actual.
    - Hay una secuencia de **estados ocultos** desconocidos que queremos inferir.
- Se compone de:
    - **Conjunto de estados ocultos (S):** Un número finito de estados que no son directamente observables. En el contexto del POS-tagging, estos estados representarían las posibles etiquetas gramaticales (sustantivo, verbo, adjetivo, etc.).
    - **Conjunto de observaciones (V):** Un número finito de símbolos de observación que son las salidas del sistema que podemos observar. En el POS-tagging, estas serían las palabras del texto que queremos etiquetar.
- Se caracteriza por 3 conjuntos de parámetros:
    - **Matriz de probabilidades de transición de estados (A):** Define la probabilidad de transición entre los estados ocultos. $a_{ij} = P(q_{t+1} = S_j | q_t = S_i)$ es la probabilidad de estar en el estado $S_i$  en el tiempo $t$ y pasar al estado $S_j$ en el tiempo $t+1$.  ***(probabilidad de pasar de una etiqueta a otra)***
    - **Matriz de probabilidades de emisión (B):** Define la probabilidad de observar un símbolo particular dado un estado oculto. $b_j(k) = P(O_t = v_k | q_t = S_j)$ es la probabilidad de observar el símbolo $v_k$ cuando el estado oculto en el tiempo $t$ es $S_j$. ***(probabilidad de que una palabra sea generada por una etiqueta.)***
    - **Vector de probabilidades iniciales de estado (π):** Define la probabilidad de estar en un estado particular en el tiempo inicial ($t=1$).
        
         $\pi_i = P(q_1 = S_i)$ es la probabilidad de que el primer estado oculto sea $S_i$. ***(probabilidad de comenzar en cada etiqueta.)***
        

**HMM de Primer Orden**

Un HMM de primer orden se caracteriza por la **propiedad de Markov de primer orden**, que establece que la probabilidad del estado actual solo depende del estado inmediatamente anterior. Matemáticamente, esto se expresa como:

$P(q_t | q_{t-1}, q_{t-2}, ..., q_1) = P(q_t | q_{t-1})$

De manera similar, la probabilidad de una observación en un momento dado solo depende del estado oculto en ese mismo momento:

$P(O_t | q_1, ..., q_t, O_1, ..., O_{t-1}) = P(O_t | q_t)$

**HMM para POS-Tagging**

En el contexto del POS-tagging, podemos mapear los componentes de un HMM de la siguiente manera:

- **Estados ocultos (S):** El conjunto de posibles etiquetas gramaticales (e.g., {NOUN, VERB, ADJ, DET}).
- **Observaciones (V):** El vocabulario del idioma (el conjunto de todas las palabras posibles).
- **Probabilidades de transición (A):** La probabilidad de que una etiqueta gramatical siga a otra (e.g., la probabilidad de que un sustantivo siga a un determinante). Estas probabilidades se pueden estimar a partir de un corpus de texto etiquetado.
- **Probabilidades de emisión (B):** La probabilidad de que una palabra particular sea generada por una etiqueta gramatical específica (e.g., la probabilidad de que la palabra "gato" tenga la etiqueta NOUN). Estas probabilidades también se pueden estimar a partir de un corpus de texto etiquetado.
- **Probabilidades iniciales (π):** La probabilidad de que una etiqueta gramatical sea la primera etiqueta en una oración (e.g., la probabilidad de que la primera palabra de una oración sea un determinante o un sustantivo).